In [50]:
# get_sepsis_score.py
#!/usr/bin/env python

import xgboost as xgb

def get_sepsis_score(data, model):

    # Get number of rows in data to know which prediction to save
    row_total = data.shape[0]

    dtest = xgb.DMatrix(data)

    # score = 1 - np.exp(-l_exp_bx)
    score = model.predict(dtest)[row_total-1]
    label = score > 0.1

    return score, label

def load_sepsis_model():

    model = xgb.Booster(model_file="xgb.model")

    return model


In [5]:
import numpy as np, os, sys
#from get_sepsis_score import load_sepsis_model, get_sepsis_score

def load_challenge_data(file):
    with open(file, 'r') as f:
        header = f.readline().strip()
        column_names = header.split('|')
        data = np.loadtxt(f, delimiter='|')

    # Ignore SepsisLabel column if present.
    if column_names[-1] == 'SepsisLabel':
        column_names = column_names[:-1]
        data = data[:, :-1]

    return data

def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability|PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g|%d\n' % (s, l))

In [9]:
input_directory = "Z:/LKS-CHART/Projects/physionet_sepsis_project/data/splits/split_0/test"
output_directory = "Z:/LKS-CHART/Projects/physionet_sepsis_project/data/training_setA/output"
files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('psv'):
        files.append(f)
        
# Load model.
print('Loading sepsis model...')
model = load_sepsis_model()


Loading sepsis model...


In [35]:
files_short = files[0:1]
files_short


list

In [41]:
    # Load data.
    input_file = os.path.join(input_directory, files_short[0]).replace("\\","/")
input_file

    data = load_challenge_data(input_file)

#     # Make predictions.
    num_rows = len(data)
    scores = np.zeros(num_rows)
    labels = np.zeros(num_rows)

In [47]:
current_data = data[:1]
current_data

array([[    nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,   70.81,
           0.  ,     nan,     nan, -223.3 ,    1.  ]])

In [51]:
current_score, current_label = get_sepsis_score(current_data, model)

In [18]:
print('Predicting sepsis labels...')
num_files = len(files_short)
for i, f in enumerate(files_short):
    print('    {}/{}...'.format(i+1, num_files))

    # Load data.
    input_file = os.path.join(input_directory, f)
    data = load_challenge_data(input_file)

    # Make predictions.
    num_rows = len(data)
    scores = np.zeros(num_rows)
    labels = np.zeros(num_rows)
    for t in range(num_rows):
        current_data = data[:t+1]
        current_score, current_label = get_sepsis_score(current_data, model)
        scores[t] = current_score
        labels[t] = current_label

    # Save results.
#     output_file = os.path.join(output_directory, f)
#     save_challenge_predictions(output_file, scores, labels)

print('Done.')

Predicting sepsis labels...
    1/11...


FileNotFoundError: [Errno 2] No such file or directory: 'Z:/LKS-CHART/Projects/physionet_sepsis_project/data/splits/split_0/test\\p'

In [4]:
# driver.py
#!/usr/bin/env python



if __name__ == '__main__':
    # Parse arguments.
    if len(sys.argv) != 3:
        raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

    input_directory = sys.argv[1]
    output_directory = sys.argv[2]

    # Find files.
    files = []
    for f in os.listdir(input_directory):
        if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('psv'):
            files.append(f)

    if not os.path.isdir(output_directory):
        os.mkdir(output_directory)

    # Load model.
    print('Loading sepsis model...')
    model = load_sepsis_model()

    # Iterate over files.
    print('Predicting sepsis labels...')
    num_files = len(files)
    for i, f in enumerate(files):
        print('    {}/{}...'.format(i+1, num_files))

        # Load data.
        input_file = os.path.join(input_directory, f)
        data = load_challenge_data(input_file)

        # Make predictions.
        num_rows = len(data)
        scores = np.zeros(num_rows)
        labels = np.zeros(num_rows)
        for t in range(num_rows):
            current_data = data[:t+1]
            current_score, current_label = get_sepsis_score(current_data, model)
            scores[t] = current_score
            labels[t] = current_label

        # Save results.
        output_file = os.path.join(output_directory, f)
        save_challenge_predictions(output_file, scores, labels)

    print('Done.')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '-f'